# BERT-модель

## Импорты библиотек

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

Определяем, на чем будет идти обучение (gpu / cpu)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Класс датасета

In [3]:
class VideoDataset(Dataset):
	def __init__(self, descriptions, tags, tokenizer, max_len, max_tags):
		self.descriptions = descriptions
		self.tags = tags
		self.tokenizer = tokenizer
		self.max_len = max_len
		self.max_tags = max_tags

	def __len__(self):
		return len(self.descriptions)

	def __getitem__(self, idx):
		description = str(self.descriptions[idx])
		tag = self.tags[idx]

		inputs = self.tokenizer(description, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

		# Вектор, где единица соответствует вхождению данного тега в множество
		tag_tensor = np.zeros(self.max_tags)
		for t in tag:
			if t < self.max_tags:
				tag_tensor[t] = 1.0

		return {
			'input_ids': inputs['input_ids'].flatten(),
			'attention_mask': inputs['attention_mask'].flatten(),
			'labels': torch.tensor(tag_tensor, dtype=torch.float)
		}

## Модель классификатора

Теги могут быть множественными, поэтому в качестве функции потерь хорошо подходит BCEWithLogitsLoss

In [4]:
class VideoTaggerModel(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = super(VideoTaggerModel, self).forward(input_ids, attention_mask=attention_mask, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
            return loss, logits

        return logits

## Вспомогательные функции для предобработки данных

In [5]:
# Предобработка подкатегории 
def cleanify_subtag(subtag):
    return subtag.strip().capitalize()


# Предобработка категории
def cleanify_tag(tag):
    return ': '.join(filter(bool, map(cleanify_subtag, tag.replace('  ', ' ').replace('\t', ':').split(':'))))


# Разбиваем строку на категории с предобработкой
def split_tags(tags):
    tags = str(tags)
    splitted_tags = list(filter(bool, map(cleanify_tag, tags.split(','))))
    return splitted_tags

In [6]:
df = pd.read_csv('train_data_categories.csv')
df

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира
...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,Массовая культура:Отношения знаменитостей: Сем...
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,Еда и напитки: Кулинария
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Спорт
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,"Книги и литература: Поэзия, Музыка и аудио, К..."


In [7]:
df['splitted_tags'] = df['tags'].apply(split_tags)
df

,video_id,title,description,tags,splitted_tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,[События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...","[Массовая культура: Отношения знаменитостей, М..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура","[Транспорт, Спорт: Автогонки, Массовая культура]"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]
...,...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,Массовая культура:Отношения знаменитостей: Сем...,[Массовая культура: Отношения знаменитостей: С...
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,Еда и напитки: Кулинария,[Еда и напитки: Кулинария]
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Спорт,[Спорт]
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,"Книги и литература: Поэзия, Музыка и аудио, К...","[Книги и литература: Поэзия, Музыка и аудио, К..."


## Находим уникальные теги

На самом деле тегов больше 600, однако используется всего чуть больше 140. Сокращение количества возможных тегов позволяет повысить качество модели 

In [8]:
unique_tags = set(sum(df['splitted_tags'].tolist(), []))
len(unique_tags)

141

## Немного аналитики

Это позволяет нам понять, что, используя метрику, IoU на маленьких данных выгоднее угадывать только один уровень, так как математическое ожидание IoU с предсказанием второго уровня будет сильно меньше. Однако, верно предсказав второй уровень, мы можем с хорошей вероятностью предсказать и третий, что повысит метрику IoU

In [9]:
tags_by_levels = {}
for tag in unique_tags:
  subtags = list(map(str.strip, tag.split(':')))
  tags_by_levels.setdefault(subtags[0], {})
  if len(subtags) > 1:
    tags_by_levels[subtags[0]].setdefault(subtags[1], {})
  if len(subtags) > 2:
    tags_by_levels[subtags[0]][subtags[1]].setdefault(subtags[2], {})


print('Тегов первого уровня', len(tags_by_levels))
print('Среднее число тегов второго уровня', np.mean([len(v) for v in tags_by_levels.values()]))
print('Среднее число тегов третьего уровня', np.mean([len(v) for k in tags_by_levels.values() for v in k.values()]))

Тегов первого уровня 36
Среднее число тегов второго уровня 2.7222222222222223
Среднее число тегов третьего уровня 0.29591836734693877


## Обрезаем тэги до первого уровня

In [10]:
def truncate_tag_to_first_level(tag):
  return str(tag).split(':')[0].strip()

def truncate_tags_to_first_level(tags):
  return list(set(map(truncate_tag_to_first_level, tags)))

In [11]:
df['truncated_splitted_tags'] = df['splitted_tags'].apply(truncate_tags_to_first_level)
df

,video_id,title,description,tags,splitted_tags,truncated_splitted_tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура]
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,[События и достопримечательности: Спортивные с...,"[События и достопримечательности, Массовая кул..."
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...","[Массовая культура: Отношения знаменитостей, М...",[Массовая культура]
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура","[Транспорт, Спорт: Автогонки, Массовая культура]","[Транспорт, Спорт, Массовая культура]"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура]
...,...,...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,Массовая культура:Отношения знаменитостей: Сем...,[Массовая культура: Отношения знаменитостей: С...,[Массовая культура]
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,Еда и напитки: Кулинария,[Еда и напитки: Кулинария],[Еда и напитки]
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Спорт,[Спорт],[Спорт]
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,"Книги и литература: Поэзия, Музыка и аудио, К...","[Книги и литература: Поэзия, Музыка и аудио, К...","[Карьера, Книги и литература, Музыка и аудио]"


## Получаем уникальные тэги первого уровня

In [12]:
unique_truncated_tags = set(map(truncate_tag_to_first_level, unique_tags))
unique_truncated_tags

{'Nan',
 'Бизнес и финансы',
 'Дом и сад',
 'Еда и напитки',
 'Здоровый образ жизни',
 'Игры',
 'Игры и головоломки',
 'Изобразительное искусство',
 'Информационные технологии',
 'Карьера',
 'Книги и литература',
 'Компьютеры и цифровые технологии',
 'Красота',
 'Личные финансы',
 'Массовая культура',
 'Медицина',
 'Медицинские направления',
 'Музыка и аудио',
 'Наука',
 'Недвижимость',
 'Новости и политика',
 'Образование',
 'Отношения знаменитостей',
 'Путешествия',
 'Религия и духовность',
 'Семья и отношения',
 'События и достопримечательности',
 'Создание контента',
 'Спорт',
 'Стиль и красота',
 'Телевидение',
 'Транспорт',
 'Фильмы и анимация',
 'Хобби и интересы',
 'Хобби и стиль',
 'Экономика'}

In [13]:
iab_tags = list(unique_truncated_tags)
iab_tags

['Спорт',
 'Дом и сад',
 'Изобразительное искусство',
 'Игры и головоломки',
 'Музыка и аудио',
 'Наука',
 'События и достопримечательности',
 'Карьера',
 'Хобби и стиль',
 'Путешествия',
 'Отношения знаменитостей',
 'Хобби и интересы',
 'Экономика',
 'Красота',
 'Семья и отношения',
 'Nan',
 'Создание контента',
 'Книги и литература',
 'Бизнес и финансы',
 'Личные финансы',
 'Здоровый образ жизни',
 'Игры',
 'Информационные технологии',
 'Стиль и красота',
 'Медицинские направления',
 'Транспорт',
 'Новости и политика',
 'Телевидение',
 'Медицина',
 'Фильмы и анимация',
 'Массовая культура',
 'Образование',
 'Еда и напитки',
 'Недвижимость',
 'Компьютеры и цифровые технологии',
 'Религия и духовность']

## Токенизация

Кодируем тэги с помощью токенизатора DeepPavlov/rubert-base-cased, который показал лучший результат при работе с русским текстом 

In [14]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
max_len = 256

tags_encoder = {tag: idx for idx, tag in enumerate(iab_tags)}
df['encoded_tags'] = df['truncated_splitted_tags'].apply(
	lambda tags: np.array([tags_encoder[tag] for tag in tags])
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Разбиение датасета

Обучаем на 90% данных, тестируем на 10%

In [15]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_dataset = VideoDataset(train_df['description'].values, train_df['encoded_tags'].tolist(), tokenizer, max_len, len(iab_tags))
test_dataset = VideoDataset(test_df['description'].values, test_df['encoded_tags'].tolist(), tokenizer, max_len, len(iab_tags))

## Модель

Используем предобученную модель DeepPavlov/rubert-base-cased

In [17]:
model = VideoTaggerModel.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=len(iab_tags))
for param in model.parameters():
    param.data = param.data.contiguous()

training_args = TrainingArguments(
   output_dir='./results',
   num_train_epochs=5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   warmup_steps=500,
   weight_decay=0.01,
   logging_dir='./logs'
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset
)

Some weights of VideoTaggerModel were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Обучение

5 эпох заняло чуть менее 4 минут. С увеличением числа эпох точность повышается

In [18]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=295, training_loss=0.3310893947795286, metrics={'train_runtime': 228.3025, 'train_samples_per_second': 20.674, 'train_steps_per_second': 1.292, 'total_flos': 621131647057920.0, 'train_loss': 0.3310893947795286, 'epoch': 5.0})

## Предсказание

Кодируем описание, которое подается на вход и по вектору вероятностей определяем, какие теги будут на выходе, отсекая по заданному порогу

In [19]:
np_iab_tags = np.array(iab_tags)

def prepare_input(description, tokenizer, max_len):
    inputs = tokenizer(description, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt')
    return inputs['input_ids'].to(device), inputs['attention_mask'].to(device)


def predict_tags(description, threshold=0.5, max_len=max_len):
    input_ids, attention_mask = prepare_input(description, tokenizer, max_len=max_len)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        if isinstance(outputs, torch.Tensor):
            logits = outputs
        else:
            logits = outputs.logits

    probabilities = torch.sigmoid(logits)
    predicted_tags = (probabilities > threshold).int()

    return np_iab_tags[predicted_tags.cpu().numpy().flatten() == 1]

In [20]:
predict_tags('Актёрское искусство. Как вести себя на кастинге. Ошибки молодых актёров.', threshold=0.43)

array(['Массовая культура'], dtype='<U32')

## Расчёт метрики

In [21]:
def expand_tag(tag: str) -> list[str]:
    subtags = tag.split(': ')
    return [': '.join(subtags[:i]) for i in range(1, len(subtags) + 1)]

def count_iou(test_tags, predicted_tags):
    set1, set2 = set(), set()
    for tag in test_tags:
        set1.update(expand_tag(tag))
    for tag in predicted_tags:
        set2.update(expand_tag(tag))
    return float(len(set1 & set2) / len(set1 | set2))

In [22]:
expand_tag('Бизнес и финансы: Промышленность и сфера услуг: Энергетическая промышленность')

['Бизнес и финансы',
 'Бизнес и финансы: Промышленность и сфера услуг',
 'Бизнес и финансы: Промышленность и сфера услуг: Энергетическая промышленность']

### Подбор порогового значения

Здесь мы подбираем теги, исходя из заданного порогового значения для их отсечения. 
Чем меньше порог, тем больше тегов (тем больше знаменатель в IoU).

In [24]:
thresholds = np.linspace(10., 50., num=9)

for th in thresholds:
    test_df[f'predicted_tags_{th}'] = test_df['description'].apply(lambda x: predict_tags(x, threshold=th / 100))
    test_df[f'iou_{th}'] = np.array([count_iou(x, y) for x, y in zip(test_df['splitted_tags'].values, test_df[f'predicted_tags_{th}'].values)])

test_df

,video_id,title,description,tags,splitted_tags,truncated_splitted_tags,encoded_tags,predicted_tags_10.0,iou_10.0,predicted_tags_20.0,...,predicted_tags_50.0,iou_50.0,predicted_tags_15.0,iou_15.0,predicted_tags_25.0,iou_25.0,predicted_tags_35.0,iou_35.0,predicted_tags_45.0,iou_45.0
483,b7b149c0dd4f9f5cea535af7e6bf95cd,Дуэль. Вспышки-малышки. Миногарова и Коваль,Выпуск 5. Битва блогеров или блогерш или блоге...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура],[30],"[События и достопримечательности, Массовая кул...",0.333333,[Массовая культура],...,[],0.000000,"[События и достопримечательности, Массовая кул...",0.333333,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000
640,ca39c17bde5cc48ce95a9a7fa5f733dc,МузЛофт-Концерт | Юлия Савичева. «Я могла сов...,"Торговый эквайринг от ПСБ за 0,9%. Возмещение ...","Массовая культура, Музыка и аудио","[Массовая культура, Музыка и аудио]","[Музыка и аудио, Массовая культура]","[4, 30]","[События и достопримечательности, Карьера, Мас...",0.200000,[Массовая культура],...,[Массовая культура],0.500000,"[События и достопримечательности, Массовая кул...",0.333333,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000
223,136669ed3a76f12ee2dff41b17ddd954,"Злые языки | Выпуск 7, Сезон 3 | Анита Цой",На этом Анита Цой собаку съела: миллионы на ог...,Массовая культура: Скандалы знаменитостей,[Массовая культура: Скандалы знаменитостей],[Массовая культура],[30],"[Спорт, События и достопримечательности, Карье...",0.200000,[Массовая культура],...,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000
29,106bd13b84f390cce9d369571604ab75,"Punch Box. 4 сезон, 7 серия. Курич Александр v...",Punch Box — бои в формате POP MMA где вместо р...,"Спорт: Борьба, Массовая культура","[Спорт: Борьба, Массовая культура]","[Спорт, Массовая культура]","[0, 30]","[Спорт, Музыка и аудио, События и достопримеча...",0.400000,[Массовая культура],...,[Массовая культура],0.333333,[Массовая культура],0.333333,[Массовая культура],0.333333,[Массовая культура],0.333333,[Массовая культура],0.333333
748,fb11e767bbc2c8fcce5ee783f4402d6a,Yunan MotoTour в ГрандТуре «Байкальская миля 2...,Команда Yunan MotoTour побывала на Красноярско...,"Путешествия, Транспорт, События и достопримеча...","[Путешествия, Транспорт, События и достопримеч...","[Транспорт, Путешествия, События и достопримеч...","[25, 9, 6]","[Спорт, События и достопримечательности, Карье...",0.125000,[Массовая культура],...,[],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,4231958ed9fc366f35afb4babf70e2bf,На связующих | Юлианна Караулова | Как найти н...,"На связующих - это шоу, в котором Кирилл Фокин...",Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[Массовая культура],[30],"[Спорт, События и достопримечательности, Карье...",0.200000,[Массовая культура],...,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000,[Массовая культура],0.500000
497,4790fc6692c29d4a50433cde975512a0,МАКСИМ НАРОДНЫЙ Выпуск №4 «ГОТОВИМ НАИВКУСНЕЙШ...,Максим Народный предлагает подписчикам пригото...,Еда и напитки: Кулинария,[Еда и напитки: Кулинария],[Еда и напитки],[32],"[Спорт, События и достопримечательности, Карье...",0.000000,[Массовая культура],...,[Массовая культура],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000,[Массовая культура],0.000000
6,f02fceb454008fe7907b47324569b845,Женсовет I Выпуск 6 I Мужчина в новой феминист...,Краткая инструкция по выживанию для мужчин в с...,Семья и отношения,[Семья и отношения],[Семья и отношения],[14],"[Спорт, События и достопримечательности, Карье...",0.000000,[Массов

## Результаты

Как можно заметить, результаты для порогов 0.20-0.40 оказались одинаковыми. Это связано с небольшой обучающей выборкой

In [25]:
[(th, test_df[f'iou_{th}'].mean()) for th in thresholds]

[(10.0, 0.19255102040816324),
 (15.0, 0.2514285714285714),
 (20.0, 0.25396825396825395),
 (25.0, 0.25396825396825395),
 (30.0, 0.25396825396825395),
 (35.0, 0.25396825396825395),
 (40.0, 0.25396825396825395),
 (45.0, 0.2515873015873016),
 (50.0, 0.20476190476190476)]

### Скорость работы

Предсказание из 264 символов занимает 30 мс

In [27]:
%time predict_tags('Нас заперли в огромном бункере и нам нужно выжить в нем 100 дней, чтобы получить миллион - 10 тысяч за каждый день! А чтобы нам было сложнее - нам устраивают разные челленджи и всячески пробуют помешать. Но мы выносливые и будем стараться изо всех сил! Ставь ЛАЙК!', threshold=0.4)

CPU times: user 28 ms, sys: 48 µs, total: 28.1 ms
Wall time: 29.3 ms


array(['Массовая культура'], dtype='<U32')

## Сохранение результирующей выборки

In [28]:
test_df.to_csv('test.csv')

## Сохранение модели

In [29]:
trainer.save_model('./model')

In [30]:
!zip -r ./model.zip ./model

  adding: model/ (stored 0%)
  adding: model/training_args.bin (deflated 51%)
  adding: model/config.json (deflated 67%)
  adding: model/model.safetensors (deflated 7%)
